# 사전 작업

In [2]:
# 필요 라이브러리 선언

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings   
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.schema import Document
import pandas as pd
import pymysql
from langchain_community.storage import SQLStore
import os
import warnings
import chromadb
warnings.filterwarnings('ignore')

In [2]:
# 필요 변수 선언

host = os.environ.get('DB_HOST')
port = 3306
username = os.environ.get('DB_USER')
password = os.getenv('DB_PASSWORD')
db = os.getenv('DB_NAME')
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# 함수 정의

def get_mysql_connection(host,port,username,password,db_name):
    try:
        conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db_name,charset='utf8',cursorclass=pymysql.cursors.DictCursor)
        print(f"MySQL 연결 성공")
        return conn
    except Exception as e:
        print(f"MySQL 연결 오류: {e}")
        return None
    
def get_document(conn,source):
    try: 
        with conn.cursor() as cursor:
            sql = f'SELECT 판례일련번호, 판례내용 FROM 판례 WHERE 판례일련번호 = {source}'
            cursor.execute(sql)
            result = cursor.fetchone()
            return result
    except Exception as e:
        print(f"MySQL 조회 오류: {e}")
        return None
    
def retrieve_db(query, vectorstore, conn):

    retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
    # print('벡터스토어 검색 중...')
    results = retriever.invoke(query)
    
    # 결과 출력
    for i, doc in enumerate(results):
        meta = doc.metadata
        # print(f"\n🔍 [결과 {i+1}]")
        # print(f"▶ 판례일련번호 : {meta['source']}")
        # print(f"▶ 사건명 : {meta['case_type']}")
        # print("유사 문단:", doc.page_content.strip())
        # result = get_document(conn,meta['source'])
        # print('▶ 전체 판례:',result['판례내용'])
        # print("\n" + "="*50)

        return doc.page_content.strip()

In [4]:
# 벡터스토어 불러오기 + mySQL 연결설정 분리

base_db_dir='./db'

print('벡터스토어 생성 중...')
vectorstore = Chroma(
    persist_directory=base_db_dir,
    embedding_function=OpenAIEmbeddings(api_key=api_key),
    collection_name='LAW_RAG'
)
print('벡터스토어 생성 완료')

conn = get_mysql_connection(host,port,username,password,db)

벡터스토어 생성 중...
벡터스토어 생성 완료
MySQL 연결 성공


# 시작

In [274]:
import openai

def generate_legal_questions(api_key):
    client = openai.OpenAI(api_key=api_key)

    prompt = """
    당신은 법률 자문 시스템을 위한 상황 데이터셋을 생성하고자 합니다.
    현실에서 사람들이 경험할 수 있는 법적으로 문제가 되는 다양한 상황을 예시로 30개 만들어주세요.
    각 상황은 3문장 내외로 구체적으로 서술되어야 하며, 형사/민사/행정 등 다양한 법적 분쟁 유형을 포함하면 좋습니다.
    출력은 아래 형식처럼 Python의 리스트 형태로 해주세요:

    [
        "나는 친구에게 500만 원을 빌려줬는데, 약속한 기한이 지나도 갚지 않고 연락을 피하고 있다. 여러 차례 연락했지만 계속 무시당하고 있다. 법적으로 대응해야 할지 고민 중이다.",
        "동네 마트에서 바닥이 젖어 있는 걸 모르고 미끄러져 크게 다쳤다. 근처에 미끄럼 주의 표지판도 없었고, 병원 진단 결과 무릎 인대가 파열되었다. 치료비와 손해배상을 요구하고 싶다.",
        ...
    ]
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
        #max_tokens=1500
    )

    # 응답에서 텍스트 추출
    generated_text = response.choices[0].message.content

    try:
        question_list = eval(generated_text.strip())  # 안전하게 하려면 ast.literal_eval 추천
    except:
        question_list = []

    return question_list

# 사용 예시
api_key = api_key  # 본인의 OpenAI API 키
questions = generate_legal_questions(api_key)

for q in questions:
    print("-", q)


- 나는 친구에게 500만 원을 빌려줬는데, 약속한 기한이 지나도 갚지 않고 연락을 피하고 있다. 여러 차례 연락했지만 계속 무시당하고 있다. 법적으로 대응해야 할지 고민 중이다.
- 동네 마트에서 바닥이 젖어 있는 걸 모르고 미끄러져 크게 다쳤다. 근처에 미끄럼 주의 표지판도 없었고, 병원 진단 결과 무릎 인대가 파열되었다. 치료비와 손해배상을 요구하고 싶다.
- 상점을 운영하는데, 갑작스럽게 임대료를 인상하겠다는 건물주의 통보를 받았다. 계약서에는 임대료 인상에 대한 내용이 명확히 명시되어 있지 않은 상황이다.
- 아파트 단지 내에서 취미로 드론을 날리다가, 이웃집 창문을 깨뜨렸다. 이웃이 창문 수리비를 요구하고 있으며, 법적 책임에 대해 알고 싶다.
- 차를 주차장에 주차해놓고 왔는데, 돌아와보니 차량이 훼손되어 있었다. CCTV를 확인해보니 주차장 관리자가 차량을 이동하다가 발생한 사고였다.
- 택시를 타고 가다가 운전자의 과속으로 인해 사고가 났다. 운전자는 거의 무사하였으나, 나는 팔을 골절하였다. 이에 대한 보상을 청구하고 싶다.
- 온라인 쇼핑몰에서 물건을 주문하였는데, 배송이 한 달 넘게 지연되고 있다. 판매자는 배송에 대한 명확한 설명이 없으며, 환불 요청에도 응답이 없다.
- 나는 공무원인데, 직장에서 성희롱을 당했다. 상사의 부적절한 말과 행동으로 인해 업무에 지장이 있으며, 계속된 상황에 대해 법적 조치를 고려하고 있다.
- 이웃집에서 매일 밤 늦게까지 큰 소리로 노래를 부른다. 이로 인해 잠을 제대로 잘 수 없는 상황이다. 이웃에게 피해를 주는 소음에 대해 법적으로 어떻게 대응할 수 있는지 알고 싶다.
- 공장에서 근무하다가, 안전 대책이 미흡한 상황에서 근무하다가 다치게 되었다. 회사는 책임을 지지 않겠다며, 병원비 지원을 거부하고 있다.
- 부동산을 매매하다가, 계약 상의 내용을 위반한 상대방으로 인해 손해를 보았다. 상대방은 계약 위반 사실을 인정하지 않고, 손해배상을 거부하고 있다.
- 직원인데, 회사에서 정당한 이유 없이 해고

In [ ]:
# 질문 생성
df = pd.DataFrame(questions, columns=['user_input'])
df.to_csv('test_files/questions.csv', encoding='utf-8-sig')

In [5]:
# 질문 불러오기
df = pd.read_csv('test_files/questions.csv', encoding='utf-8-sig')

questions = df['user_input']

contexts = []

for i in questions:
    # print(f'질문: {i}')
    contexts.append(retrieve_db(i, vectorstore, conn))  
    # contexts.append("'hello'")
    

In [19]:
df['response'] = contexts

df.to_csv('test_files/questions_retrieve.csv', encoding='utf-8-sig')

In [20]:
df = pd.read_csv('test_files/questions_retrieve.csv', encoding='utf-8-sig')
df = df[['user_input', 'response']]

df.head()

,user_input,response
0,"나는 친구에게 500만 원을 빌려줬는데, 약속한 기한이 지나도 갚지 않고 연락을 피...","다. 피청구인은 청구인이 위 박○○에게 임금을 지불하지 않았다고 주장하나, 청구인은..."
1,동네 마트에서 바닥이 젖어 있는 걸 모르고 미끄러져 크게 다쳤다. 근처에 미끄럼 주...,적의 파편에 맞아 우측 다리에 부상을 입고 ○○지구경찰병원 ○○분소에서 치료받은 사...
2,"상점을 운영하는데, 갑작스럽게 임대료를 인상하겠다는 건물주의 통보를 받았다. 계약서...",고용ㆍ인사관계에 있어서도 이직전 사업장과 밀접한 관련성이 있다고 할 수 없는 점 등...
3,"아파트 단지 내에서 취미로 드론을 날리다가, 이웃집 창문을 깨뜨렸다. 이웃이 창문 ...",소리를 지르고 짜증을 내어 아파트 경비원이 나와 볼 정도였다고 청구인의 불친절행위를...
4,"차를 주차장에 주차해놓고 왔는데, 돌아와보니 차량이 훼손되어 있었다. CCTV를 확...","것 같아 영업을 쉬기로 하고, 길옆에 주차하여 두었던 청구인의 개인택시를 옆 골목으..."


In [39]:
# !pip install datasets
# !pip install ragas

In [21]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['user_input', 'response'],
    num_rows: 29
})

In [37]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        #context_precision,
        #faithfulness,
        answer_relevancy,
        #context_recall,
    ],
)

result


Evaluating:   0%|          | 0/29 [00:00<?, ?it/s]

{'answer_relevancy': 0.2881}

In [10]:
result_df = result.to_pandas()
result_df.to_csv('test_files/result.csv', encoding='utf-8-sig')

In [258]:
# !pip install pandasgui

In [11]:
from pandasgui import show

result_df = pd.read_csv('test_files/result.csv', encoding='utf-8-sig')
show(result_df.head(30))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


# 코사인 유사도 평가

In [23]:
# !pip install sentence_transformers

In [30]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 1. CSV 불러오기
df = pd.read_csv("test_files/questions_retrieve.csv")

# 2. 모델 로딩 (한국어에 적합한 SBERT 모델 추천)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 3. 코사인 유사도 계산 함수
def compute_cosine_similarity(row):
    query_emb = model.encode(row['user_input'], convert_to_tensor=True)
    context_emb = model.encode(row['response'], convert_to_tensor=True)
    return util.cos_sim(query_emb, context_emb).item()

# 4. 전체 행에 대해 적용
df['cos_acc'] = df.apply(compute_cosine_similarity, axis=1)

df = df.sort_values(by='cos_acc', ascending=False)

# 5. 결과 확인
df = df[['user_input', 'response', 'cos_acc']]

# 6. 저장 (선택)
df.to_csv("test_files/questions_retrieve_with_cosine.csv", index=False)

In [3]:
from pandasgui import show

df = pd.read_csv('test_files/questions_retrieve_with_cosine.csv', encoding='utf-8-sig')
show(df.head(30))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


# LLM 평가

In [35]:
import pandas as pd
import openai

# 최신 openai 라이브러리용 클라이언트 설정
client = openai.OpenAI(api_key=api_key)

# CSV 불러오기
df = pd.read_csv("test_files/questions_retrieve.csv")

# 프롬프트 템플릿
def build_prompt(user_input, response):
    return f"""
### 사용자 질문:
{user_input}

### RAG로 검색된 문단:
{response}

### 평가 기준:
주어진 문단이 사용자 질문과 얼마나 잘 일치하는지 0~10 사이의 점수로 평가해 주세요.

0점: 전혀 일치하지 않거나, 문단 내용과 모순됨  
4점: 부분적으로 일치하지만 중요한 오차 또는 누락이 있음  
7점: 대체로 일치하지만 경미한 오류나 누락이 있음  
10점: 완전히 일치하고 정확함

숫자 점수만 출력하세요. (예: 7)
"""

# LLM 점수 평가 함수 (openai>=1.0)
def get_llm_score(user_input, response):
    prompt = build_prompt(user_input, response)
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        score_text = chat_completion.choices[0].message.content.strip()
        score = int(score_text.split()[0])  # 숫자만 추출
        return score
    except Exception as e:
        print(f"Error: {e}")
        return None

# 상위 5개 샘플만 테스트 (비용 고려)
df_sample = df[['user_input', 'response']]
df_sample['LLM_score'] = df_sample.apply(lambda row: get_llm_score(row['user_input'], row['response']), axis=1)

# 결과 저장
df_sample.to_csv("test_files/llm_scored_output.csv", index=False)
print(df_sample[['user_input', 'LLM_score']])


                                           user_input  LLM_score
0   나는 친구에게 500만 원을 빌려줬는데, 약속한 기한이 지나도 갚지 않고 연락을 피...          0
1   동네 마트에서 바닥이 젖어 있는 걸 모르고 미끄러져 크게 다쳤다. 근처에 미끄럼 주...          0
2   상점을 운영하는데, 갑작스럽게 임대료를 인상하겠다는 건물주의 통보를 받았다. 계약서...          0
3   아파트 단지 내에서 취미로 드론을 날리다가, 이웃집 창문을 깨뜨렸다. 이웃이 창문 ...          0
4   차를 주차장에 주차해놓고 왔는데, 돌아와보니 차량이 훼손되어 있었다. CCTV를 확...          4
5   택시를 타고 가다가 운전자의 과속으로 인해 사고가 났다. 운전자는 거의 무사하였으나...          0
6   온라인 쇼핑몰에서 물건을 주문하였는데, 배송이 한 달 넘게 지연되고 있다. 판매자는...          0
7   나는 공무원인데, 직장에서 성희롱을 당했다. 상사의 부적절한 말과 행동으로 인해 업...          0
8   이웃집에서 매일 밤 늦게까지 큰 소리로 노래를 부른다. 이로 인해 잠을 제대로 잘 ...          0
9   공장에서 근무하다가, 안전 대책이 미흡한 상황에서 근무하다가 다치게 되었다. 회사는...          4
10  부동산을 매매하다가, 계약 상의 내용을 위반한 상대방으로 인해 손해를 보았다. 상대...          0
11  직원인데, 회사에서 정당한 이유 없이 해고 당했다. 이에 대한 법적 대응을 고려하고...          2
12  주차장에 차를 주차해 놓았는데, 돌아오니 차량이 도난당한 상태였다. 주차장 측은 C...          0
13  아이가 놀이터에서 미끄럼틀에서 미끄러지다가 크게 다쳤다. 미끄럼틀 설치가 불안정하고...          0
14  집을 살 때 중개업자가 중요한 정보를 

# 기타 지표

In [39]:
# !pip install rouge_score

In [47]:
from rouge_score import rouge_scorer
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

df = pd.read_csv("test_files/questions_retrieve.csv")

sent1 = df['user_input'][:5]
sent2 = df['response'][:5]

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

for i in range(len(sent1)):
    print(
        f"[1] {sent1[i]}\n[2] {sent2[i]}\n[rouge1] {scorer.score(sent1[i], sent2[i])['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1[i], sent2[i])['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1[i], sent2[i])['rougeL'].fmeasure:.5f}"
    )
    print("===" * 20)

[1] 나는 친구에게 500만 원을 빌려줬는데, 약속한 기한이 지나도 갚지 않고 연락을 피하고 있다. 여러 차례 연락했지만 계속 무시당하고 있다. 법적으로 대응해야 할지 고민 중이다.
[2] 다. 피청구인은 청구인이 위 박○○에게 임금을 지불하지 않았다고 주장하나, 청구인은 위 박○○을 채용함에 있어 근로계약서를 작성하였고 매월 고정적으로 임금을 지불하진 않았으나 월 100만원의 목돈을 한번에 지급하기엔 사정이 여의치 않아 필요에 따라 수시로 가불해 주는 형태로 임금을 지급하였다.
[rouge1] 0.35294
[rouge2] 0.07947
[rougeL] 0.22222
[1] 동네 마트에서 바닥이 젖어 있는 걸 모르고 미끄러져 크게 다쳤다. 근처에 미끄럼 주의 표지판도 없었고, 병원 진단 결과 무릎 인대가 파열되었다. 치료비와 손해배상을 요구하고 싶다.
[2] 적의 파편에 맞아 우측 다리에 부상을 입고 ○○지구경찰병원 ○○분소에서 치료받은 사실이 있다고 인우보증하고 있는 점 등에 비추어 볼 때, 이 건 처분은 위법ㆍ부당하여 취소되어야 한다고 주장한다.
[rouge1] 0.28829
[rouge2] 0.03670
[rougeL] 0.16216
[1] 상점을 운영하는데, 갑작스럽게 임대료를 인상하겠다는 건물주의 통보를 받았다. 계약서에는 임대료 인상에 대한 내용이 명확히 명시되어 있지 않은 상황이다.
[2] 고용ㆍ인사관계에 있어서도 이직전 사업장과 밀접한 관련성이 있다고 할 수 없는 점 등으로 미루어 볼 때, 청구인이 이직전 사업의 시설ㆍ설비 또는 그 임차권을 유상 또는 무상으로 양도받은 사업주에 해당된다는 이유로 채용장려금의 지급을 거부한 피청구인의 이 건 처분은 위법ㆍ부당하다고 할 것이다.
[rouge1] 0.25758
[rouge2] 0.03077
[rougeL] 0.15152
[1] 아파트 단지 내에서 취미로 드론을 날리다가, 이웃집 창문을 깨뜨렸다. 이웃이 창문 수리비를 요구하고 있으며, 법적 책임에 대해 알고 싶다.
[2] 소리를 지르고 짜증을 내어 